### Overview:

In [2]:
from os import putenv
putenv("HSA_OVERRIDE_GFX_VERSION", "11.0.0") # The line must be defined before importing torch.

import torch # OK. The HIP Runtime of PyTorch can recognize your ISA.
import torch.nn as nn

#### Import PDF Document:

In [4]:
import os
import requests

pdf_path = 'Essentials of Anatomy and Physiology.pdf'


In [ ]:
import 